In [2]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
import implicit
from sklearn.preprocessing import MinMaxScaler

In [3]:
articles_df = pd.read_csv('shared_articles.csv')
interactions_df = pd.read_csv('users_interactions.csv')
articles_df.drop(['authorUserAgent', 'authorRegion', 'authorCountry'], axis=1, inplace=True)
interactions_df.drop(['userAgent', 'userRegion', 'userCountry'], axis=1, inplace=True)


In [4]:
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.drop('eventType', axis=1, inplace=True)
df = pd.merge(interactions_df[['contentId','personId', 'eventType']], articles_df[['contentId', 'title']], how = 'inner', on = 'contentId')

df['eventType'].value_counts()

VIEW               61043
LIKE                5745
BOOKMARK            2463
COMMENT CREATED     1611
FOLLOW              1407
Name: eventType, dtype: int64

In [5]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 3.0, 
   'FOLLOW': 4.0,
   'COMMENT CREATED': 5.0,  
}
df['eventStrength'] = df['eventType'].apply(lambda x: event_type_strength[x])

In [6]:
grouped_df = df.groupby(['personId', 'contentId', 'title']).sum().reset_index()
grouped_df.sample(10)

,personId,contentId,title,eventStrength
16240,-1443636648652872475,-4974757204495953627,Alecrim/AlecrimCoreData,1.0
36531,7047489233409645826,991271693336573226,Este é o melhor jeito de entender como as mulh...,3.0
20151,-108842214936804958,4575580212020245658,Conversational Banking; from branches to bots....,1.0
10689,-3596626804281480007,18738895644634365,Bot do Facebook Messenger simula entrevista pa...,5.0
34183,5621833459783231486,5338677278233757627,How to Get a Job at Google,1.0
1893,-8620763856232712186,9124439338148818380,"How to build a robot that ""sees"" with $100 and...",1.0
26676,2785512916786763092,6044362651232258738,Cinco competências comportamentais para você s...,1.0
16437,-1443636648652872475,1286723199785027310,The Skills You'll Need and the Salary You Can ...,2.0
12708,-2979537012405607453,-2059465483219605309,"[Free Online Course] Sit Less, Get Active - Th...",1.0
38393,8051175896420731576,-8950763842769120954,Mobify report reveals impact of mobile website...,1.0


In [7]:
grouped_df['title'] = grouped_df['title'].astype("category")
grouped_df['personId'] = grouped_df['personId'].astype("category")
grouped_df['contentId'] = grouped_df['contentId'].astype("category")
grouped_df['person_id'] = grouped_df['personId'].cat.codes
grouped_df['content_id'] = grouped_df['contentId'].cat.codes
